#### garrett r peternel | senior data scientist | lm

https://docs.aws.amazon.com/sagemaker/latest/dg/randomcutforest.html

https://aws.amazon.com/blogs/machine-learning/use-the-built-in-amazon-sagemaker-random-cut-forest-algorithm-for-anomaly-detection/

# aws sagemaker Rcf

In [ ]:
print("="*100)

# mods

In [ ]:
import boto3
import botocore
import sagemaker
import sys
import pandas as pd
import numpy as np

# sagemaker integration

In [ ]:
sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()

s3_bucket = 'aero-iiot'  
s3_prefix = 'datasets/e363549/sagemaker/sgMkr-Rcf-Ac1'
data_key = '6f82cf81-d1c7-4041-9151-5bbde3ba5876.csv'

s3_data_path = "s3://{}/{}/data".format(s3_bucket, s3_prefix)
s3_output_path = "s3://{}/{}/output".format(s3_bucket, s3_prefix)

# data

In [ ]:
ac1_df = pd\
.read_csv("s3://aero-iiot/datasets/e363549/queries/6f82cf81-d1c7-4041-9151-5bbde3ba5876.csv",\
          delimiter=',')

In [ ]:
ac1_df['timestamp'] = pd.to_datetime(ac1_df['timestamp']).dt.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
min(ac1_df.timestamp), max(ac1_df.timestamp)

# clean

In [ ]:
ac1_df.fillna(method='ffill', inplace=True)
ac1_df.fillna(method='bfill', inplace=True)

# limit for inference pipeline performance

In [ ]:
ac1_df = ac1_df.tail(100000).reset_index(drop=True)

In [ ]:
ac1_df.head()

In [ ]:
min(ac1_df.timestamp), max(ac1_df.timestamp)

In [ ]:
ac1_df.count()

# plot

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.dpi'] = 100

ac1_df.plot()

# fit

In [ ]:
from sagemaker import RandomCutForest

session = sagemaker.Session()

rcf = RandomCutForest(role=role,
                      train_instance_count=1,
                      train_instance_type='ml.m4.xlarge',
                      base_job_name='sgMkr-Rcf-Ac1',
                      data_location=s3_data_path,
                      output_path=s3_output_path,
                      num_samples_per_tree=512,
                      num_trees=50)

# uploads data to s3 input path
rcf.fit(rcf.record_set(ac1_df.ac1_event_autoclavetemp.as_matrix().reshape(-1,1)))

In [ ]:
print('Training job name: {}'.format(rcf.latest_training_job.job_name))

# inference

In [ ]:
rcf_inference = rcf.deploy(
    initial_instance_count=4,
    instance_type='ml.m4.4xlarge'
)

In [ ]:
print('Endpoint name: {}'.format(rcf_inference.endpoint))

In [ ]:
rcf_inference.delete_endpoint()

# data serialize/deserialize

In [ ]:
from sagemaker.predictor import csv_serializer, json_deserializer

rcf_inference.content_type = 'text/csv'
rcf_inference.serializer = csv_serializer
rcf_inference.accept = 'application/json'
rcf_inference.deserializer = json_deserializer

# test endpoint

In [ ]:
ac1_data_numpy = ac1_df.ac1_event_autoclavetemp.as_matrix().reshape(-1,1)
print(ac1_data_numpy[:10])
results = rcf_inference.predict(ac1_data_numpy[:10])

# calculate scores on inference pipeline

In [ ]:
results = rcf_inference.predict(ac1_data_numpy)
scores = [i['score'] for i in results['scores']]

ac1_df['score'] = pd.Series(scores, index=ac1_df.index)

In [ ]:
ac1_df.head()

# plot

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

#
# *Try this out* - change `start` and `end` to zoom in on the 
# anomaly found earlier in this notebook
#
#start, end = 0, len(ac1_df)
#start, end = 5500, 6500
#ac1_subset = ac1_df[start:end]

ax1.plot(ac1_df['ac1_event_autoclavetemp'], color='C0', alpha=0.8)
ax2.plot(ac1_df['score'], color='C1')

ax1.grid(which='major', axis='both')

ax1.set_ylabel('Temp', color='C0')
ax2.set_ylabel('Anomaly Score', color='C1')

ax1.tick_params('y', colors='C0')
ax2.tick_params('y', colors='C1')

ax1.set_ylim(min(ac1_subset.ac1_event_autoclavetemp), max(ac1_subset.ac1_event_autoclavetemp)*1.5)
ax2.set_ylim(min(scores), 1.5*max(scores))
fig.set_figwidth(10)

# anomalies

In [ ]:
score_mean = ac1_df['score'].mean()
score_std = ac1_df['score'].std()
score_cutoff = score_mean + 3*score_std

anomalies = ac1_df[ac1_df['score'] > score_cutoff]

In [ ]:
score_cutoff

In [ ]:
anomalies.count()

In [ ]:
ax2.plot(anomalies.index, anomalies.score, 'ko')
fig

#### garrett r peternel | senior data scientist | lm